## Tensorflow

In [30]:
import numpy as np
import torch
import tensorflow as tf
import torch.nn.functional


### Def des fonctions 

In [5]:
def tf_convert_points_to_homogeneous(points):
    """Function that converts points from Euclidean to homogeneous space.

    See :class:`~torchgeometry.ConvertPointsToHomogeneous` for details.

    Examples::

        >>> input = torch.rand(2, 4, 3)  # BxNx3
        >>> output = tgm.convert_points_to_homogeneous(input)  # BxNx4
    """
    if not tf.is_tensor(points):
        raise TypeError("Input type is not a tf.Tensor. Got {}".format(
            type(points)))
    if len(points.shape) < 2:
        raise ValueError("Input must be at least a 2D tensor. Got {}".format(
            points.shape))
    paddings = tf.constant([[0,0], [0,1]])
    
    return tf.pad(points, paddings, mode='CONSTANT', constant_values=1)


def tf_symmetrical_epipolar_distance(pts1,pts2,Fm,  squared= True, eps = 1e-8) : 
    """
    Return symmetrical epipolar distance for correspondences given the fundamental matrix.

    """
    
    if not isinstance(Fm, tf.Tensor):
        raise TypeError(f"Fm type is not a torch.Tensor. Got {type(Fm)}")

    if (len(Fm.shape) != 3) or not Fm.shape[-2:] == (3, 3):
        raise ValueError(f"Fm must be a (*, 3, 3) tensor. Got {Fm.shape}")

        ###########Acompleter
    if pts1.size(-1) == 2:
        pts1 = convert_points_to_homogeneous(pts1)

    if pts2.size(-1) == 2:
        pts2 = convert_points_to_homogeneous(pts2)

    # From Hartley and Zisserman, symmetric epipolar distance (11.10)
    # sed = (x'^T F x) ** 2 /  (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    # Instead we can just transpose F once and switch the order of multiplication
    
    F_t = tf.transpose(Fm, perm=(0,2,1), conjugate=False, name='permute')
    line1_in_2 = pts1 @ F_t
    line2_in_1 = pts2 @ Fm

    # numerator = (x'^T F x) ** 2
    #numerator  = (pts2 * line1_in_2).sum(2).pow(2)
    numerator  = tf.pow(tf.math.reduce_sum((pts2 * line1_in_2),2),2)


    # denominator_inv =  1/ (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    denominator_inv = 1.0 / (line1_in_2[..., :2].norm(2, dim=2).pow(2)) + 1.0 / (
        line2_in_1[..., :2].norm(2, dim=2).pow(2)
    )
    
    denominator_inv = 1.0 / (tf.pow(tf.norm(line1_in_2[..., :2],axis=2),2)) + 1.0 / (
        tf.pow(tf.norm(line2_in_1[..., :2],axis=2),2)
    )
    
    out = numerator * denominator_inv
    if squared:
        return out
    return tf.math.sqrt(out + eps)

def fundamental_matrix_from_projections(P1, P2):
    """
    Get the Fundamental matrix from Projection matrices.
    Adapted from 
    [
    https://kornia.readthedocs.io/en/latest/_modules/kornia/
    geometry/epipolar/fundamental.html
    ]
    """
    if not (len(P1.shape) >= 2 and P1.shape[-2:] == (3, 4)):
        raise AssertionError(P1.shape)
    if not (len(P2.shape) >= 2 and P2.shape[-2:] == (3, 4)):
        raise AssertionError(P2.shape)
    if P1.shape[:-2] != P2.shape[:-2]:
        raise AssertionError

    def vstack(x, y):
            return tf.concat([x,y], axis=0, name='concat')
    X1 = P1[..., 1:, :]
    X2 = vstack(P1[..., 2:3, :], P1[..., 0:1, :])
    X3 = P1[..., :2, :]

    Y1 = P2[..., 1:, :]
    Y2 = vstack(P2[..., 2:3, :], P2[..., 0:1, :])
    Y3 = P2[..., :2, :]

    X1Y1, X2Y1, X3Y1 = vstack(X1, Y1), vstack(X2, Y1), vstack(X3, Y1)
    X1Y2, X2Y2, X3Y2 = vstack(X1, Y2), vstack(X2, Y2), vstack(X3, Y2)
    X1Y3, X2Y3, X3Y3 = vstack(X1, Y3), vstack(X2, Y3), vstack(X3, Y3)
    F_vec = tf.concat(
        [
            tf.reshape(tf.linalg.det(X1Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X1Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X1Y3),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y3),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y3),shape=(-1,1)),
        ],
        axis=-1
    )

    return tf.reshape(F_vec,shape=(*P1.shape[:-2],3,3))


#### On prend deux matrices de projection de deux camera de Campus_seq et on calcule F


In [43]:
# Matrice 1
P1t=tf.constant([[439.0,180.81,-26.946,185.95],[-5.3416,88.523,-450.95,1324],[0.0060594,0.99348,-0.11385,5.227]])
# Matrice 2
P2t=tf.constant([[162.36,-438.34,-17.508,3347.4],[73.3,-10.043,-443.34,1373.5],[0.99035,-0.047887,-0.13009,6.6849]])
# Calcul de la matrice fondamentale
ft=fundamental_matrix_from_projections(P1t,P2t)
ft.numpy()

array([[-1.20740205e+04,  1.59010288e+06, -1.49354128e+08],
       [ 1.91498150e+06,  6.74619453e+04,  3.05496512e+08],
       [-1.50978688e+08, -1.10211149e+09,  7.75774781e+10]], dtype=float32)


### On vérifie $x=PX$  
avec X coord 3D et x sa projection 2D
#### On choisit un point 3D et on cherche sa projection sur les deux cameras

In [51]:
pt3Dt = tf.constant([[2.9872, 4.0063, 0.1581]])
# passage en coord homogens (rajout d'un 1)
pt3D_ht=tf_convert_points_to_homogeneous(pt3Dt)
# projections
# camera 2
x2t=tf.matmul(P2t,tf.transpose(pt3D_ht, perm=(1,0)))
x2t=x2t/x2t[2]
print(x2t)
# camera 1
x1t=tf.matmul(P1t,tf.transpose(pt3D_ht, perm=(1,0)))
x1t=x1t/x1t[2]
print(x1t)


tf.Tensor(
[[219.86467]
 [157.15797]
 [  1.     ]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[240.8366 ]
 [172.84128]
 [  1.     ]], shape=(3, 1), dtype=float32)


#### Ces deux valeurs doivent être égales à [220,158,1] et [ 241  ,172,1]               (groudTruth de VoxelPose)

### On vérifie $x^t F x' = 0$  
avec x' et x deux projections 2D
#### On choisit un point 3D et on cherche sa projection sur les deux cameras

In [52]:
tf.matmul(tf.matmul(tf.transpose(x2t, perm=(1,0)),ft),x1t).numpy()

array([[90112.]], dtype=float32)

=> Why ?!

# Pytorch

### Def des fonction

In [42]:
# Ici on compare la méthode implémentée sur tensorflow à la méthode déja existante de la library kornia

# Kornia : 


import torch
import tensorflow as tf
# Version kornia
# version kornia
import torch.nn.functional

def convert_points_to_homogeneous(points):
    r"""Function that converts points from Euclidean to homogeneous space.

    See :class:`~torchgeometry.ConvertPointsToHomogeneous` for details.

    Examples::

        >>> input = torch.rand(2, 4, 3)  # BxNx3
        >>> output = tgm.convert_points_to_homogeneous(input)  # BxNx4
    """
    if not torch.is_tensor(points):
        raise TypeError("Input type is not a torch.Tensor. Got {}".format(
            type(points)))
    if len(points.shape) < 2:
        raise ValueError("Input must be at least a 2D tensor. Got {}".format(
            points.shape))

    return torch.nn.functional.pad(points, (0, 1), "constant", 1.0)
def symmetrical_epipolar_distance(
    pts1: torch.Tensor, pts2: torch.Tensor, Fm: torch.Tensor, squared: bool = True, eps: float = 1e-8
) -> torch.Tensor:
    r"""Return symmetrical epipolar distance for correspondences given the fundamental matrix.

    Args:
       pts1: correspondences from the left images with shape
         (B, N, 2 or 3). If they are not homogeneous, converted automatically.
       pts2: correspondences from the right images with shape
         (B, N, 2 or 3). If they are not homogeneous, converted automatically.
       Fm: Fundamental matrices with shape :math:`(B, 3, 3)`. Called Fm to
         avoid ambiguity with torch.nn.functional.
       squared: if True (default), the squared distance is returned.
       eps: Small constant for safe sqrt.

    Returns:
        the computed Symmetrical distance with shape :math:`(B, N)`.

    """
    if not isinstance(Fm, torch.Tensor):
        raise TypeError(f"Fm type is not a torch.Tensor. Got {type(Fm)}")

    if (len(Fm.shape) != 3) or not Fm.shape[-2:] == (3, 3):
        raise ValueError(f"Fm must be a (*, 3, 3) tensor. Got {Fm.shape}")

    if pts1.size(-1) == 2:
        pts1 = convert_points_to_homogeneous(pts1)

    if pts2.size(-1) == 2:
        pts2 = convert_points_to_homogeneous(pts2)

    # From Hartley and Zisserman, symmetric epipolar distance (11.10)
    # sed = (x'^T F x) ** 2 /  (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))

    # line1_in_2: torch.Tensor = (F @ pts1.permute(0,2,1)).permute(0,2,1)
    # line2_in_1: torch.Tensor = (F.permute(0,2,1) @ pts2.permute(0,2,1)).permute(0,2,1)

    # Instead we can just transpose F once and switch the order of multiplication
    F_t: torch.Tensor = Fm.permute(0, 2, 1).type(torch.LongTensor)
    print(pts1)
    line1_in_2 = pts1 @ F_t
    line2_in_1: torch.Tensor = pts2 @ Fm

    # numerator = (x'^T F x) ** 2
    numerator: torch.Tensor = (pts2 * line1_in_2).sum(2).pow(2)

    # denominator_inv =  1/ (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    denominator_inv: torch.Tensor = 1.0 / (line1_in_2[..., :2].norm(2, dim=2).pow(2)) + 1.0 / (
        line2_in_1[..., :2].norm(2, dim=2).pow(2)
    )
    out: torch.Tensor = numerator * denominator_inv
    if squared:
        return out
    return (out + eps).sqrt()


def fundamental_from_projections(P1: torch.Tensor, P2: torch.Tensor) -> torch.Tensor:
    r"""Get the Fundamental matrix from Projection matrices.

    Args:
        P1: The projection matrix from first camera with shape :math:`(*, 3, 4)`.
        P2: The projection matrix from second camera with shape :math:`(*, 3, 4)`.

    Returns:
         The fundamental matrix with shape :math:`(*, 3, 3)`.

    """
    def vstack(x, y):
        return torch.cat([x, y], dim=-2)
    X1 = P1[..., 1:, :]
    X2 = vstack(P1[..., 2:3, :], P1[..., 0:1, :])
    X3 = P1[..., :2, :]

    Y1 = P2[..., 1:, :]
    Y2 = vstack(P2[..., 2:3, :], P2[..., 0:1, :])
    Y3 = P2[..., :2, :]

    X1Y1, X2Y1, X3Y1 = vstack(X1, Y1), vstack(X2, Y1), vstack(X3, Y1)
    X1Y2, X2Y2, X3Y2 = vstack(X1, Y2), vstack(X2, Y2), vstack(X3, Y2)
    X1Y3, X2Y3, X3Y3 = vstack(X1, Y3), vstack(X2, Y3), vstack(X3, Y3)
    F_vec = torch.cat(
        [
            X1Y1.det().reshape(-1, 1),
            X2Y1.det().reshape(-1, 1),
            X3Y1.det().reshape(-1, 1),
            X1Y2.det().reshape(-1, 1),
            X2Y2.det().reshape(-1, 1),
            X3Y2.det().reshape(-1, 1),
            X1Y3.det().reshape(-1, 1),
            X2Y3.det().reshape(-1, 1),
            X3Y3.det().reshape(-1, 1),
        ],
        dim=1,
    )
    return F_vec.view(*P1.shape[:-2], 3, 3)


def triangulate_points(
    P1: torch.Tensor, P2: torch.Tensor, points1: torch.Tensor, points2: torch.Tensor
) -> torch.Tensor:
    r"""Reconstructs a bunch of points by triangulation.

    Triangulates the 3d position of 2d correspondences between several images.
    Reference: Internally it uses DLT method from Hartley/Zisserman 12.2 pag.312

    The input points are assumed to be in homogeneous coordinate system and being inliers
    correspondences. The method does not perform any robust estimation.

    Args:
        P1: The projection matrix for the first camera with shape :math:`(*, 3, 4)`.
        P2: The projection matrix for the second camera with shape :math:`(*, 3, 4)`.
        points1: The set of points seen from the first camera frame in the camera plane
          coordinates with shape :math:`(*, N, 2)`.
        points2: The set of points seen from the second camera frame in the camera plane
          coordinates with shape :math:`(*, N, 2)`.

    Returns:
        The reconstructed 3d points in the world frame with shape :math:`(*, N, 3)`.

    """
    if not (len(P1.shape) >= 2 and P1.shape[-2:] == (3, 4)):
        raise AssertionError(P1.shape)
    if not (len(P2.shape) >= 2 and P2.shape[-2:] == (3, 4)):
        raise AssertionError(P2.shape)
    if len(P1.shape[:-2]) != len(P2.shape[:-2]):
        raise AssertionError(P1.shape, P2.shape)
    if not (len(points1.shape) >= 2 and points1.shape[-1] == 2):
        raise AssertionError(points1.shape)
    if not (len(points2.shape) >= 2 and points2.shape[-1] == 2):
        raise AssertionError(points2.shape)
    if len(points1.shape[:-2]) != len(points2.shape[:-2]):
        raise AssertionError(points1.shape, points2.shape)
    if len(P1.shape[:-2]) != len(points1.shape[:-2]):
        raise AssertionError(P1.shape, points1.shape)

    # allocate and construct the equations matrix with shape (*, 4, 4)
    print('_______________________________')
    points_shape = max(points1.shape, points2.shape)  # this allows broadcasting
    X = torch.zeros(points_shape[:-1] + (4, 4)).type_as(points1)
    print(X)
    for i in range(4):
        X[..., 0, i] = points1[..., 0] * P1[..., 2:3, i] - P1[..., 0:1, i]
        X[..., 1, i] = points1[..., 1] * P1[..., 2:3, i] - P1[..., 1:2, i]
        X[..., 2, i] = points2[..., 0] * P2[..., 2:3, i] - P2[..., 0:1, i]
        X[..., 3, i] = points2[..., 1] * P2[..., 2:3, i] - P2[..., 1:2, i]

    # 1. Solve the system Ax=0 with smallest eigenvalue
    # 2. Return homogeneous coordinates

    _, _, V = torch.svd(X)

    points3d_h = V[..., -1]
    points3d: torch.Tensor = convert_points_from_homogeneous(points3d_h)
    return points3d


def convert_points_from_homogeneous(points: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    r"""Function that converts points from homogeneous to Euclidean space.

    Args:
        points: the points to be transformed of shape :math:`(B, N, D)`.
        eps: to avoid division by zero.

    Returns:
        the points in Euclidean space :math:`(B, N, D-1)`.

    Examples:
        >>> input = torch.tensor([[0., 0., 1.]])
        >>> convert_points_from_homogeneous(input)
        tensor([[0., 0.]])
    """
    if not isinstance(points, torch.Tensor):
        raise TypeError(f"Input type is not a torch.Tensor. Got {type(points)}")

    if len(points.shape) < 2:
        raise ValueError(f"Input must be at least a 2D tensor. Got {points.shape}")

    # we check for points at max_val
    z_vec: torch.Tensor = points[..., -1:]

    # set the results of division by zeror/near-zero to 1.0
    # follow the convention of opencv:
    # https://github.com/opencv/opencv/pull/14411/files
    mask: torch.Tensor = torch.abs(z_vec) > eps
    scale = torch.where(mask, 1.0 / (z_vec + eps), torch.ones_like(z_vec))

    return scale * points[..., :-1]


#### On prend deux matrices de projection de deux camera de Campus_seq et on calcule F



In [47]:
# Matrice 1
P1=torch.tensor([[439.0,180.81,-26.946,185.95],[-5.3416,88.523,-450.95,1324],[0.0060594,0.99348,-0.11385,5.227]])
# Matrice 2
P2=torch.tensor([[162.36,-438.34,-17.508,3347.4],[73.3,-10.043,-443.34,1373.5],[0.99035,-0.047887,-0.13009,6.6849]])
# Calcul de la matrice fondamentale
f=fundamental_from_projections(P1,P2)
f.numpy()

array([[-1.2074024e+04,  1.5901038e+06, -1.4935422e+08],
       [ 1.9149804e+06,  6.7461953e+04,  3.0549645e+08],
       [-1.5097872e+08, -1.1021116e+09,  7.7577454e+10]], dtype=float32)

### On vérifie $x=PX$  
avec X coord 3D et x sa projection 2D
#### On choisit un point 3D et on cherche sa projection sur les deux cameras

In [56]:
pt3D = torch.tensor([[2.9872, 4.0063, 0.1581]])
# passage en coord homogens (rajout d'un 1)
pt3D_h=convert_points_to_homogeneous(pt3D)
# projections
# camera 2
x2=torch.matmul(P2,torch.transpose(pt3D_h, 0, 1))
x2=x2/x2[2]
print(x2)
# camera 1
x1=torch.matmul(P1,torch.transpose(pt3D_h, 0, 1))
x1=x1/x1[2]
print(x1)


tensor([[219.8647],
        [157.1580],
        [  1.0000]])
tensor([[240.8366],
        [172.8413],
        [  1.0000]])


#### Ces deux valeurs doivent être égales à [220,158,1] et [ 241  ,172,1]               (groudTruth de VoxelPose)
### On vérifie $x^t F x' = 0$  
avec x' et x deux projections 2D
#### On choisit un point 3D et on cherche sa projection sur les deux cameras

In [57]:
torch.matmul(torch.matmul(torch.transpose(x2,1,0),f),x1).numpy()

array([[0.]], dtype=float32)

=> YESSSSS ! 

# L'erreur sur tf ??

In [83]:
## x2^t F x1  donne pas zéro
tf.matmul(tf.matmul(tf.transpose(x2,(1,0)),f),x1).numpy()

array([[-16384.]], dtype=float32)

### la valeur de x2^t

In [85]:
tf.transpose(x2,(1,0)).numpy()

array([[219.86467, 157.15799,   1.     ]], dtype=float32)

### si on arrandit ...

In [86]:
x2t=tf.constant([[219.8647, 157.1580,   1.0000 ]])

tf.matmul(tf.matmul(x2t,f),x1).numpy()

array([[0.]], dtype=float32)